In [1]:
import requests
from bs4 import BeautifulSoup

In [40]:
_BASE = 'https://doegen.ie/counties'
def do_get(url):
    r = requests.get(url, headers = {'User-agent': 'Mozilla/5.0'})
    if r.status_code != 200:
        raise Exception("Failed to open landing page")
    return r.content

In [10]:
soup = BeautifulSoup(do_get(_BASE), 'html.parser')

In [26]:
counties = soup.find('ul', {'class': 'vocabindex'}).find_all('li')

In [37]:
pages = []
for county in counties:
    item = {}
    anchor = county.find('a')
    href = anchor['href']
    item['link'] = f'https://doegen.ie{href}'
    if anchor.find('span').text.strip() != '(0)':
        item['county'] = anchor.text.split()[1]
        pages.append(item)

In [64]:
def proc_page(url):
    result = {}
    html = do_get(url)
    soup = BeautifulSoup(html, 'html.parser')
    content = soup.find('div', {'id': 'main'}).find('div', {'class': 'content'})
    result['mp3'] = content.find('source')['src']
    result['transcript'] = content.find('div', id='transcript').text
    result['translation'] = content.find('div', id='translation').text
    result['footnote'] = content.find('div', id='footnote').text
    result['recording_metadata'] = content.find('div', id='recording_metadata').text
    return result

In [65]:
def proc_county(item):
    content = do_get(item['link'])
    soup = BeautifulSoup(content, 'html.parser')
    main = soup.find('div', id='main')
    nodes = main.find_all('div', {'class': 'node'})
    stories = []
    for node in nodes:
        story = {}
        anchor = node.find('a')
        story['link'] = f"https://doegen.ie{anchor['href']}"
        story['content'] = proc_page(story['link'])
        tags = node.find('div', {'class': 'terms'})
        text = anchor.text
        if ' - ' in text:
            tmp = text.split(' - ')
            if len(tmp) == 2:
                story['title'] = tmp[0]
                story['speaker_name'] = tmp[1]
            else:
                story['raw'] = text
        else:
            story['raw'] = text
        stories.append(story)
    item['stories'] = stories

In [ ]:
proc_page('https://doegen.ie/LA_1051d3')